<a href="https://colab.research.google.com/github/AsmaaYassinDev/Behavioural-Anomaly-Detection-for-ATO-Fraud/blob/main/ATO_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
import warnings

# Ignore unimportant warnings
warnings.filterwarnings('ignore', category=UserWarning)

print("--- Execution Started (Stable Code) ---")
print("Objective: Identify Anomalous Accounts based on their behavior.")

# --- Load Data (Make sure this name matches the file you uploaded) ---
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded the full file ({len(df)} rows).")
except Exception as e:
    print(f"Error during data loading: {e}")
    print("!!! Make sure the file name in the code (file_path) matches the file in the folder exactly !!!")
    exit()

# --- Step 1: Create the 'Smart Sample' ---
print("\n--- Step 1: Creating the 'Smart Sample' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]

df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)

df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"The final 'Smart Sample' was created with {len(df_smart_sample)} rows.")

# --- Step 2: Build Behavioral Profiles (Strong Features) ---
print("\n--- Step 2: Building Behavioral Profiles ---")

# (a) Calculate total received, total cashed out, and unique senders count
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()

df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)

    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0) # The ratio cannot exceed 100%

    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })

final_profiles = pd.DataFrame(profiles_list)
print("Behavioral profiles created successfully.")

# --- Step 3: Merge Features with Transactions ---
print("\n--- Step 3: Merging Features with Transactions ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)

# --- Step 4: Train an Unsupervised Model ---
print("\n--- Step 4: Training Isolation Forest Model ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest', # Recipient's cash-out ratio
    'dest_unique_senders_dest', # Recipient's unique senders
    'dest_cash_out_ratio_orig', # Sender's cash-out ratio
    'dest_unique_senders_orig'  # Sender's unique senders
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # The "Correct Answer"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

contamination = y_true.mean()
print(f"Fraud (Contamination) rate in the Smart Sample: {contamination:.2%}")

model = IsolationForest(contamination=contamination, random_state=42)
model.fit(X_scaled)
predictions = model.predict(X_scaled) # -1 = anomalous, 1 = normal
print("Model training complete.")

# --- Step 5: Answer Your Question ---
print("\n--- Step 5: Identifying Anomalous Accounts ---")

# Add the model's "guess" to the data
# 1 = anomalous, 0 = normal
df_model_data['anomaly_prediction'] = [1 if p == -1 else 0 for p in predictions]

# Find the "transactions" that the model judged as anomalous
anomalous_transactions = df_model_data[df_model_data['anomaly_prediction'] == 1]

# Find the "account names" (senders and recipients) involved in these anomalous transactions
anomalous_dest_accounts = anomalous_transactions['nameDest'].unique()
anomalous_orig_accounts = anomalous_transactions['nameOrig'].unique()

all_anomalous_accounts = np.union1d(anomalous_dest_accounts, anomalous_orig_accounts)

print(f"\n[Final Result]: The model found {len(all_anomalous_accounts)} 'anomalous' accounts.")

# Print a sample of 20 accounts the model considered 'anomalous'
print("Sample of accounts the model considered 'anomalous':")
print(all_anomalous_accounts[:20])

# --- For Comparison: What are the "Real" Fraudulent Accounts? ---
print("\n--- For Comparison (The Truth) ---")
print(f"The number of 'real' fraudulent accounts (isFraud=1) was: {len(all_fraud_user_ids)}")
print("Sample of 'real' fraudulent accounts:")
print(all_fraud_user_ids[:20])

# Calculate F1-Score to verify quality
f1 = f1_score(y_true, df_model_data['anomaly_prediction'])
print(f"\nThe model's F1-Score (for confirmation): {f1:.2%}")

print("\n--- Stable Code Execution Complete ---")

--- Execution Started (Stable Code) ---
Objective: Identify Anomalous Accounts based on their behavior.
Successfully loaded the full file (1048575 rows).

--- Step 1: Creating the 'Smart Sample' ---
The final 'Smart Sample' was created with 504964 rows.

--- Step 2: Building Behavioral Profiles ---
Behavioral profiles created successfully.

--- Step 3: Merging Features with Transactions ---

--- Step 4: Training Isolation Forest Model ---
Fraud (Contamination) rate in the Smart Sample: 0.23%
Model training complete.

--- Step 5: Identifying Anomalous Accounts ---

[Final Result]: The model found 1950 'anomalous' accounts.
Sample of accounts the model considered 'anomalous':
['C1000820773' 'C1000839468' 'C1001444586' 'C10015111' 'C1003526443'
 'C1003775387' 'C1007717381' 'C1008947638' 'C1010765614' 'C1011097249'
 'C1013511446' 'C1013700132' 'C1014154376' 'C1015888357' 'C1016521533'
 'C1017653240' 'C1018394275' 'C1021713645' 'C102174220' 'C1022269511']

--- For Comparison (The Truth) ---

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
# --- (Fix: Imported the correct model name) ---
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import warnings

# Ignore unimportant warnings
warnings.filterwarnings('ignore', category=UserWarning)

print("--- Execution Started (Smartest Code: With 'Time' Feature) ---")
print("Objective: Improve F1-Score by adding 'avg_time_to_cash_out'.")

# --- Load Data ---
# Make sure this name matches the file you uploaded
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded the full file ({len(df)} rows).")
except Exception as e:
    print(f"Error during data loading: {e}")
    exit()

# --- Step 1: Create the 'Smart Sample' ---
print("\n--- Step 1: Creating the 'Smart Sample' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]
df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)
df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"The final 'Smart Sample' was created with {len(df_smart_sample)} rows.")

# --- Step 2: Building Behavioral Profiles (Including 'Time') ---
print("\n--- Step 2: Building Behavioral Profiles (Full) ---")

# (a) Calculate basic stats (Ratio, Sender Count)
print("   (a) Calculating cash-out ratio and unique senders...")
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()
df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)
    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0)
    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })
final_profiles = pd.DataFrame(profiles_list)

# (b) Calculate average time to cash-out (the smartest feature using 'step')
print("   (b) Calculating average time to cash-out (avg_time_to_cash_out)...")
df_transfers = df_smart_sample[df_smart_sample['type'] == 'TRANSFER'][['step', 'nameDest']]
df_cashouts = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT'][['step', 'nameOrig']]
df_transfers.rename(columns={'nameDest': 'user_id'}, inplace=True)
df_cashouts.rename(columns={'nameOrig': 'user_id'}, inplace=True)
df_transfers['tx_type'] = 'TRANSFER_IN'
df_cashouts['tx_type'] = 'CASH_OUT'

user_log = pd.concat([df_transfers, df_cashouts]).sort_values(by=['user_id', 'step'])
user_log['prev_step'] = user_log.groupby('user_id')['step'].shift(1)
user_log['prev_type'] = user_log.groupby('user_id')['tx_type'].shift(1)

# (Correcting the previous error)
user_log['time_since_transfer'] = user_log['step'] - user_log['prev_step']
is_pattern = (user_log['tx_type'] == 'CASH_OUT') & (user_log['prev_type'] == 'TRANSFER_IN')
pattern_times = user_log[is_pattern]

avg_time_profile = pattern_times.groupby('user_id')['time_since_transfer'].mean().reset_index()
avg_time_profile.columns = ['user_id', 'avg_time_to_cash_out']

# (c) Aggregate final profiles
print("   (c) Aggregating final profiles...")
final_profiles = pd.merge(final_profiles, avg_time_profile, on='user_id', how='left')
# Fill NaNs: If the user didn't follow the pattern, set a long time (e.g., 999)
final_profiles['avg_time_to_cash_out'] = final_profiles['avg_time_to_cash_out'].fillna(999)
print("Complex behavioral profiles created successfully.")

# --- Step 3: Merge Features with Transactions ---
print("\n--- Step 3: Merging Features with Transactions ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

# Fill NaNs from the merge
df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['avg_time_to_cash_out_dest'] = df_model_data['avg_time_to_cash_out_dest'].fillna(999)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)
df_model_data['avg_time_to_cash_out_orig'] = df_model_data['avg_time_to_cash_out_orig'].fillna(999)

# --- Step 4: Train Unsupervised Model ---
print("\n--- Step 4: Training Isolation Forest Model (With New Features) ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest',
    'dest_unique_senders_dest',
    'avg_time_to_cash_out_dest', # <-- New feature (for recipient)
    'dest_cash_out_ratio_orig',
    'dest_unique_senders_orig',
    'avg_time_to_cash_out_orig'  # <-- New feature (for sender)
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # The "Correct Answer" (for evaluation only)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

contamination = y_true.mean()
print(f"Fraud (Contamination) rate in the Smart Sample: {contamination:.2%}")

# --- (This is the corrected line) ---
model = IsolationForest(contamination=contamination, random_state=42)
model.fit(X_scaled)
predictions = model.predict(X_scaled) # -1 = anomalous, 1 = normal
print("Model training complete.")

# --- Step 5: Evaluate the Model (Comparison) ---
print("\n--- Step 5: Evaluating the Model (New Score) ---")

y_pred = [1 if p == -1 else 0 for p in predictions]
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-Score (New Final Score): {f1:.2%}")
print("\nConfusion Matrix:")
print(cm)

print("\n--- Smartest Code Execution Complete ---")

--- Execution Started (Smartest Code: With 'Time' Feature) ---
Objective: Improve F1-Score by adding 'avg_time_to_cash_out'.
Successfully loaded the full file (1048575 rows).

--- Step 1: Creating the 'Smart Sample' ---
The final 'Smart Sample' was created with 504964 rows.

--- Step 2: Building Behavioral Profiles (Full) ---
   (a) Calculating cash-out ratio and unique senders...
   (b) Calculating average time to cash-out (avg_time_to_cash_out)...
   (c) Aggregating final profiles...
Complex behavioral profiles created successfully.

--- Step 3: Merging Features with Transactions ---

--- Step 4: Training Isolation Forest Model (With New Features) ---
Fraud (Contamination) rate in the Smart Sample: 0.23%
Model training complete.

--- Step 5: Evaluating the Model (New Score) ---
Precision: 3.13%
Recall: 3.06%
F1-Score (New Final Score): 3.10%

Confusion Matrix:
[[502740   1082]
 [  1107     35]]

--- Smartest Code Execution Complete ---


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
# --- (Change: We will use a Supervised model) ---
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
# --- (Change: We need to split the data) ---
from sklearn.model_selection import train_test_split
import warnings

# Ignore unimportant warnings
warnings.filterwarnings('ignore', category=UserWarning)

print("--- Execution Started (Successful Model: Supervised) ---")
print("Objective: Prove that 'Behavioral Features' succeed with a Supervised model.")

# --- Load Data ---
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded the full file ({len(df)} rows).")
except Exception as e:
    print(f"Error during data loading: {e}")
    exit()

# --- Step 1: Create the 'Smart Sample' ---
# (Same code to build a balanced sample for training)
print("\n--- Step 1: Creating the 'Smart Sample' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]
df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)
df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"The final 'Smart Sample' was created with {len(df_smart_sample)} rows.")

# --- Step 2: Building Behavioral Profiles (Strong Features) ---
print("\n--- Step 2: Building Behavioral Profiles ---")
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()
df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)
    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0)
    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })
final_profiles = pd.DataFrame(profiles_list)
print("Behavioral profiles created successfully.")

# --- Step 3: Merge Features with Transactions ---
print("\n--- Step 3: Merging Features with Transactions ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)

# --- Step 4: Prepare Train/Test Data (Supervised) ---
print("\n--- Step 4: Preparing Train/Test Data ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest', # Recipient's cash-out ratio
    'dest_unique_senders_dest', # Recipient's unique senders
    'dest_cash_out_ratio_orig', # Sender's cash-out ratio
    'dest_unique_senders_orig'  # Sender's unique senders
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # The "Correct Answer"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# (Change: We split the data 70% for training and 30% for testing)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_true, test_size=0.3, random_state=42, stratify=y_true)
print(f"Data split into {len(X_train)} rows for training and {len(X_test)} rows for testing.")

# --- Step 5: Train Supervised RandomForest Model ---
print("\n--- Step 5: Training RandomForest Model (Supervised) ---")

# (class_weight='balanced' is very important for imbalanced data)
model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# Test on "new" data (X_test) that the model has never seen before
predictions = model.predict(X_test)
print("Model training complete.")

# --- Step 6: Evaluate (The Successful Result) ---
print("\n--- Step 6: Evaluating the Model (New Result) ---")

f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
cm = confusion_matrix(y_test, predictions)

print("!!! Results on 'Test Data' (data the model has never seen before) !!!")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-Score (New Final Score): {f1:.2%}")
print("\nConfusion Matrix:")
print(cm)

# --- (The most important step: Why did the model succeed?) ---
print("\n--- Feature Importance (Why did the model succeed?) ---")
feature_imp = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print(feature_imp)

print("\n--- Successful Code Execution Complete ---")

--- Execution Started (Successful Model: Supervised) ---
Objective: Prove that 'Behavioral Features' succeed with a Supervised model.
Successfully loaded the full file (1048575 rows).

--- Step 1: Creating the 'Smart Sample' ---
The final 'Smart Sample' was created with 504964 rows.

--- Step 2: Building Behavioral Profiles ---
Behavioral profiles created successfully.

--- Step 3: Merging Features with Transactions ---

--- Step 4: Preparing Train/Test Data ---
Data split into 353474 rows for training and 151490 rows for testing.

--- Step 5: Training RandomForest Model (Supervised) ---
Model training complete.

--- Step 6: Evaluating the Model (New Result) ---
!!! Results on 'Test Data' (data the model has never seen before) !!!
Precision: 27.03%
Recall: 20.41%
F1-Score (New Final Score): 23.26%

Confusion Matrix:
[[150958    189]
 [   273     70]]

--- Feature Importance (Why did the model succeed?) ---
amount                      0.539584
type_encoded                0.234897
dest_u